# Classificação (adaptado)

Notebook adaptado para: tratamento de desbalanceamento, métricas robustas, validação com GridSearchCV, calibração de probabilidades, tratamento básico de outliers e ajuste do pipeline polinomial.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, roc_auc_score, classification_report,
    confusion_matrix, precision_recall_curve, average_precision_score,
    roc_curve
)
from sklearn.calibration import CalibratedClassifierCV

# Opcional: imbalanced-learn para SMOTE
# !pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

pd.set_option('display.max_columns', None)
plt.style.use('seaborn')


In [ ]:
# Carregar o dataframe (ajuste o caminho se necessário)
df = pd.read_parquet('df_final')
df.shape


In [ ]:
df.head()


In [ ]:
# Verificar proporção de classes
print('Distribuição da variável alvo:')
print(df['SeriousDlqin2yrs'].value_counts())
print('
Proporção de positivos (1):', df['SeriousDlqin2yrs'].mean())


## Feature engineering / tratamento simples de outliers
Transformações simples para reduzir skew e outliers extremos.

In [ ]:
df_proc = df.copy()

# Tratar MonthlyIncome com log1p (evita problema com zero)
df_proc['MonthlyIncome_log'] = np.log1p(df_proc['MonthlyIncome'].clip(lower=0))

# Limitar DebtRatio e DebtPerLoan para reduzir outliers extremos (ex.: clip)
df_proc['DebtRatio_clipped'] = df_proc['DebtRatio'].clip(upper=10)  # ajuste se necessario
df_proc['DebtPerLoan_clipped'] = df_proc['DebtPerLoan'].clip(upper=100)  # ajuste se necessario

# Remover colunas originais muito problemáticas (mantemos as transformadas)
cols_to_use = [
    'RevolvingUtilizationOfUnsecuredLines', 'age', 'NumberOfTime30-59DaysPastDueNotWorse',
    'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines',
    'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents', 'IncomePerDependent',
    'Pagamentos_atrasados_Total', 'MonthlyIncome_log', 'DebtRatio_clipped', 'DebtPerLoan_clipped'
]

X = df_proc[cols_to_use]
y = df_proc['SeriousDlqin2yrs']

X.shape


In [ ]:
# Divisão treino/teste com stratify
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
print('Train:', X_train.shape, 'Test:', X_test.shape)


## Definir pipelines
Construí 3 pipelines:
- Regressão logística (linear)
- Naive Bayes (Gaussian)
- Regressão logística com interações polinomiais (degree=2) — PolynomialFeatures antes do scaler

Incluí também exemplo com SMOTE (opcional) usando imblearn Pipeline — lembre-se: SMOTE deve ser aplicado apenas no treino.

In [ ]:
pipe_logreg = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=5000, solver='saga', class_weight='balanced', random_state=42))
])

pipe_nb = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GaussianNB())
])

# Polinomial: gerar polinômios primeiro, depois escalar os atributos expandidos
pipe_poly = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=5000, solver='saga', class_weight='balanced', random_state=42))
])

# Exemplo opcional com SMOTE (usar ImbPipeline para garantir que SMOTE ocorra dentro do CV corretamente se desejado)
pipe_logreg_smote = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=5000, solver='saga', random_state=42))
])


## GridSearchCV para otimizar regularização (C) e, se quiser, degree do polinômio
Usamos StratifiedKFold para evitar vazamento de proporções de classe.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_logreg = {
    'model__C': [0.01, 0.1, 1, 10]
}
param_poly = {
    'poly__degree': [2],  # ja definido; se quiser testar, colocar [2,3]
    'model__C': [0.01, 0.1, 1, 10]
}

gscv_logreg = GridSearchCV(pipe_logreg, param_logreg, scoring='roc_auc', cv=cv, n_jobs=-1)
gscv_nb = GridSearchCV(pipe_nb, {'model__var_smoothing': [1e-9, 1e-8, 1e-7]}, scoring='roc_auc', cv=cv, n_jobs=-1)
gscv_poly = GridSearchCV(pipe_poly, param_poly, scoring='roc_auc', cv=cv, n_jobs=-1)

models_cv = {
    'Logística': gscv_logreg,
    'NaiveBayes': gscv_nb,
    'LogísticaPolinomial': gscv_poly
}

resultados = []

for nome, gscv in models_cv.items():
    print(f'Fazendo CV e GridSearch para: {nome}')
    gscv.fit(X_train, y_train)
    print('Melhor params:', gscv.best_params_, 'Best CV ROC-AUC:', gscv.best_score_)
    resultados.append({'Modelo': nome, 'BestParams': gscv.best_params_, 'CV_ROC_AUC': gscv.best_score_})

resultados_df = pd.DataFrame(resultados)
display(resultados_df)


## Avaliação final no conjunto de teste
Usamos o melhor estimador encontrado pelo GridSearch para cada modelo, calculamos várias métricas e plotamos ROC e Precision-Recall.

In [ ]:
evals = []

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
for nome, gscv in models_cv.items():
    best = gscv.best_estimator_
    # Calibrar probabilidades para modelos que não as emitem bem (opcional)
    clf = CalibratedClassifierCV(best, cv=3) if nome!='NaiveBayes' else best
    try:
        clf.fit(X_train, y_train)
    except Exception:
        # se o best ja estiver calibrado ou ocorrer erro, usa o best diretamente
        clf = best

    # Previsoes e probabilidades
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)[:,1] if hasattr(clf, 'predict_proba') else None

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob) if y_prob is not None else np.nan
    ap = average_precision_score(y_test, y_prob) if y_prob is not None else np.nan
    report = classification_report(y_test, y_pred, output_dict=True)

    evals.append({
        'Modelo': nome,
        'Acurácia': acc,
        'ROC_AUC': auc,
        'PR_AUC (AP)': ap,
        'Precision': report['1']['precision'] if '1' in report else None,
        'Recall': report['1']['recall'] if '1' in report else None,
        'F1': report['1']['f1-score'] if '1' in report else None
    })

    # ROC curve
    if y_prob is not None:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.plot(fpr, tpr, label=f'{nome} (AUC={auc:.3f})')

plt.plot([0,1],[0,1],'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()

plt.subplot(1,2,2)
for nome, gscv in models_cv.items():
    best = gscv.best_estimator_
    clf = CalibratedClassifierCV(best, cv=3) if nome!='NaiveBayes' else best
    try:
        clf.fit(X_train, y_train)
    except Exception:
        clf = best
    y_prob = clf.predict_proba(X_test)[:,1] if hasattr(clf, 'predict_proba') else None
    if y_prob is not None:
        prec, rec, _ = precision_recall_curve(y_test, y_prob)
        ap = average_precision_score(y_test, y_prob)
        plt.plot(rec, prec, label=f'{nome} (AP={ap:.3f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend()
plt.tight_layout()
plt.show()

evals_df = pd.DataFrame(evals)
display(evals_df)


In [ ]:
# Mostrar matrizes de confusão e classification_report detalhado para o melhor modelo (exemplo)
best_name = evals_df.sort_values('ROC_AUC', ascending=False).iloc[0]['Modelo']
best_gscv = models_cv[best_name]
best_est = best_gscv.best_estimator_
clf = CalibratedClassifierCV(best_est, cv=3) if best_name!='NaiveBayes' else best_est
try:
    clf.fit(X_train, y_train)
except Exception:
    clf = best_est

y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:,1] if hasattr(clf, 'predict_proba') else None

print('Melhor modelo:', best_name)
print('
Classification report:')
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title(f'Matriz de Confusão - {best_name}')
plt.show()


## Observações finais e próximos passos
- Se o dataset for muito desbalanceado (ex.: < 5% positivos), prefira otimizar por PR-AUC (average_precision) e olhar recall/precision no limiar escolhido.
- Teste SMOTE (ou undersampling / combinação) dentro de um pipeline com CV para evitar leak. Use ImbPipeline quando incluir SMOTE.
- Experimente regularização (C), penalidades L1/L2 e seleção de features (feature selection) para reduzir overfitting, especialmente com PolynomialFeatures.
- Verifique possíveis vazamentos ao criar features (garanta que foram feitas apenas com informações disponíveis no momento da previsão).
- Calibração é importante se você precisa de probabilidades bem calibradas para decisões (ex.: score de crédito com thresholds comerciais).

Se quiser, eu adapto o notebook ainda mais: implemento GridSearch/RandomizedSearch incluindo SMOTE dentro do CV (ImbPipeline), ou eu submeto este notebook de volta ao repositório em uma nova branch com commit. Qual opção prefere?